In [2]:
# 필요한 패키지들을 설치합니다. Huggingface의 Transformers, Accelerate, datasets, torch, pandas, scikit-learn, numpy 등이 포함됩니다.
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q torch
!pip install pandas
!pip install scikit-learn
!pip install numpy
# CUDA 12.1을 사용하는 PyTorch 설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        # 입력 텍스트와 레이블, 토크나이저, 최대 길이를 초기화합니다.
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        # 데이터셋의 길이를 반환합니다.
        return len(self.texts)

    def __getitem__(self, index):
        # 주어진 인덱스에 해당하는 텍스트와 레이블을 가져옵니다.
        text = self.texts[index]
        label = self.labels[index]

        # 텍스트를 토크나이저를 사용해 인코딩합니다.
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()  # input_ids 텐서를 얻고 차원을 줄입니다.
        attention_mask = encoding['attention_mask'].squeeze()  # attention_mask 텐서를 얻고 차원을 줄입니다.

        # 인코딩된 input_ids, attention_mask, 레이블을 반환합니다.
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}


C:\Users\Admin\anaconda3\envs\kobert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

# 데이터셋 로드
df = pd.read_csv("merge_crawling_data2.csv", encoding='cp949')

# 'data'와 'target' 열에 NaN 값이 있는 행을 제거합니다.
df.dropna(subset=['data', 'target'], inplace=True)

# 텍스트와 레이블 리스트를 만듭니다.
texts = df['data'].tolist()
labels = df['target'].tolist()

# NaN 값 제거 및 float 타입을 정수형으로 변환합니다.
clean_texts = []
clean_labels = []

for text, label in zip(texts, labels):
    if not np.isnan(label):  # 레이블이 NaN이 아닌 경우
        clean_texts.append(text)
        clean_labels.append(int(label))  # 정수형으로 변환하여 추가

# 클린된 텍스트와 레이블을 사용합니다.
texts = clean_texts
labels = clean_labels


In [3]:
# KcBERT 모델과 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 원하는 최대 시퀀스 길이
max_length = 256

# 레이블을 텐서로 변환
labels = torch.tensor(labels, dtype=torch.long)
# 커스텀 데이터셋을 만듭니다.
dataset = CustomDataset(texts, labels, tokenizer, max_length)

# Train / Test set 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.15, random_state=42)

# 훈련 및 검증 데이터셋을 만듭니다.
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)

# 데이터 로더를 만듭니다.
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 디바이스 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 하이퍼파라미터 설정
learning_rate = 1e-5
epochs = 3

# 옵티마이저 및 손실 함수 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# 모델 재학습
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 전환
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # 모델에 입력을 주어 예측을 생성합니다.
        outputs = model(input_ids, attention_mask=attention_mask)
        # 모델 출력에서 로짓(분류에 대한 점수)을 얻습니다.
        logits = outputs.logits
        # 손실을 계산합니다.
        loss = criterion(logits, labels)
        # 역전파를 통해 그래디언트 계산
        loss.backward()
        # 옵티마이저를 사용해 가중치를 업데이트
        optimizer.step()
        # 에포크 전체 손실을 누적합니다.
        total_loss += loss.item()

    # 에포크 평균 손실 계산
    avg_loss = total_loss / len(train_dataloader)
    # 에포크별 손실 출력
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.4f}")

    # 모델 평가
    model.eval()  # 모델을 평가 모드로 전환
    val_total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # 평가 시 그래디언트 계산 비활성화
        for val_batch in valid_dataloader:
            # Validation 데이터 가져오기
            val_input_ids = val_batch['input_ids']
            val_attention_mask = val_batch['attention_mask']
            val_labels = val_batch['label']

            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            # 모델 예측
            val_outputs = model(val_input_ids, attention_mask=val_attention_mask)
            val_logits = val_outputs.logits

            # 손실 계산
            val_loss = criterion(val_logits, val_labels)
            val_total_loss += val_loss.item()

            # 정확도 계산
            val_preds = val_logits.argmax(dim=1)
            correct += (val_preds == val_labels).sum().item()
            total += val_labels.size(0)

    val_avg_loss = val_total_loss / len(valid_dataloader)
    val_accuracy = correct / total
    print(f"Validation Loss: {val_avg_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")


C:\Users\Admin\anaconda3\envs\kobert\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Admin\anaconda3\envs\kobert\lib\site-packages\transformers\models\bert\modeling_bert.py:437: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch 1/3 - Avg Loss: 0.2454
Validation Loss: 0.0915 - Validation Accuracy: 0.9740
Epoch 2/3 - Avg Loss: 0.0707
Validation Loss: 0.0474 - Validation Accuracy: 0.9851
Epoch 3/3 - Avg Loss: 0.0227
Validation Loss: 0.0439 - Validation Accuracy: 0.9888


In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}

def valid_label(label):
    if label == 0:
        return '마약'
    else:
        return '평서문'

# 모델과 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 입력 데이터 준비
input_data = [
    "아이스작대기 텔레 @blueseal1004 북한산술텔상담후 인증채널 확인! 아이스샘플 훅오는 아이스작대기 블루씰과 상담하세요. 테스트 완료되지 않은 물건 판매 안합니다. 무조건 최상 똥술 절대 판매 안합니다. 알만한 사람은 다 아는 블루씰 무조건 안전! ?? 텔레 @blueseal1004작대기 #아이스 #작대기아이스 #아이스작대기 #빙두 #아이스술 #작대기술 #북한산술 #북한산아이스 #시원한술 #차가운술 #찬술 #케이캔디 #케타민 #암페타민 #후리 #사끼 #원석케이 #작대기하는법 #아이스하는법 #액상대마 #허브액상 #작대기팔아요 #아이스팔아요 #크리스탈술 #얼음술 #코카인 #필로폰 #아이스판매 #아이스구매 #아이스후기 #아이스하는법 #브액 #허브액상 #액상대마 #아이스팔아요 #아이스사는곳",
    "【텔레 @blueseal1004】아이스판매 24시간상담! 아이스작대기 전국드랍텔상담후 인증채널 확인! 아이스샘플 훅오는 아이스작대기 블루씰과 상담하세요. 테스트 완료되지 않은 물건 판매 안합니다. 무조건 최상 똥술 절대 판매 안합니다. 알만한 사람은 다 아는 블루씰 무조건 안전! ?? 텔레 @blueseal1004작대기 #아이스 #작대기아이스 #아이스작대기 #빙두 #아이스술 #작대기술 #북한산술 #북한산아이스 #시원한술 #차가운술 #찬술 #케이캔디 #케타민 #암페타민 #후리 #사끼 #원석케이 #작대기하는법 #아이스하는법 #작대기팔아요 #아이스팔아요 #크리스탈술 #얼음술 #코카인 #필로폰 #아이스판매 #아이스구매 #아이스후기 #아이스하는법 #브액 #허브액상 #액상대마 #아이스팔아요 #아이스사는곳 #크리스탈얼음술 #눈꽃 #물뽕",
    "아이스작대기 ??텔레문의?-?@MESSIAH66안전한 다크메시아! 언제나 안전보장 시장통 퀄과는 다른 아이스 똥술 멍술과는 비교불가! 뿅가는 아이스 대,소량 전국드랍 완료! ??텔레문의 - @MESSIAH66작대기 #아이스 #작대기아이스 #아이스작대기 #브액 #빙두 #아이스캔디 #버드 #허브레드 #떨액 #원석케이 #신종마약 #뉴케이 #대마젤리 #LSD스위스 #아이스술 #작대기술 #시원한술 #차가운술 #찬술 #케이캔디 #케타민 #졸피뎀 #떨 #암페타민 #후리 #사끼 #대마 #크리스탈술 #얼음술 #코카인 #필로폰 #아이스구매 #작대기구매 #작대기사요 #아이스사요 #빙두구매 #허브 #작대기삼 #아이스삼 #빙두삼 #빙두판매합니다 #작대기판매 #아이스판매 #작대기구매하는법 #아이스구매하는법 #작대기구매처 #작대기후기 #아이스후기 #아이스구매처 #빙두구매처 #브액구매",
    "같은 세대에서 가격 대 성능비가 압도적인 GPU들 위주로 작성하며, 그냥 성능이 압도적인 최상위 GPU의 작성은 피하고 시대순으로 작성할 것. 참고로 가격은 세월이나 수급 문제 등으로 유동적일 수 있으니 가성비의 기간에도 유의할 것. 예를 들면, 코인 열풍이 불었던 2017년 말~2018년 초, 2020년 말~2021년 초에는 대다수의 GPU 가격이 폭등해버려 본 문서가 무의미해졌다. 2020년 말에 채굴붐이 나고 2022년 중순쯤부터 상황이 점점 안정되다가 9월 15일에 이더리움 채굴 방식이 바뀌었고, AMD 그래픽 카드는 안정되었지만, NVIDIA 그래픽 카드는 신경망 AI 붐이 뒤따르면서 2023년 12월 기준으로도 여전히 무의미한 상황이다.",
    "진짜 북한산 빙두(필로폰을 뜻하는 은어)예요. 순도 99.9%의 투명도입니다.마약의 주요 유통 경로로 꼽히는 '텔레그램'에서 판매책이 활개를 친다. 경찰 추적이 어려운 해외 메신저의 특성을 이용해 마약 투약 후기와 불법 촬영물, 수사를 피하는 방법 등을 버젓이 공유하는 것이다. 텔레그램에서는 마약을 들고 도망갔다면서 특정인의 사진과 신상정보를 공개하기까지 한다." ,
    "북한사람들이 빙두를 하는 이유는 다양합니다. 먼저 의료가 일천한 북한에서는 빙두가 응급처치약으로 쓰이곤 합니다. 뇌혈전이나 일산화탄소 중독으로 쓰러질 경우 빙두를 하면 살 수 있다는 인식이 퍼져있어서 주로 노인들이 집에 1~2그램씩 가지고 있다는 것입니다. 뿐만 아니라 빙두가 각종 질환을 고치는데 효과가 있다고 믿는 사람도 있습니다. 마약을 약으로 사용하는 이들은 빙두가 중독되지 않는다고 굳게 믿고 있습니다. 이러한 잘못된 인식은 빙두가 북한 사회에 퍼지는데 결정적인 역할을 하고 있습니다.북한사람들이 빙두를 하는 이유는 다양합니다. 먼저 의료가 일천한 북한에서는 빙두가 응급처치약으로 쓰이곤 합니다. 뇌혈전이나 일산화탄소 중독으로 쓰러질 경우 빙두를 하면 살 수 있다는 인식이 퍼져있어서 주로 노인들이 집에 1~2그램씩 가지고 있다는 것입니다. 뿐만 아니라 빙두가 각종 질환을 고치는데 효과가 있다고 믿는 사람도 있습니다. 마약을 약으로 사용하는 이들은 빙두가 중독되지 않는다고 굳게 믿고 있습니다. 이러한 잘못된 인식은 빙두가 북한 사회에 퍼지는데 결정적인 역할을 하고 있습니다.북한사람들이 빙두를 하는 이유는 다양합니다. 먼저 의료가 일천한 북한에서는 빙두가 응급처치약으로 쓰이곤 합니다. 뇌혈전이나 일산화탄소 중독으로 쓰러질 경우 빙두를 하면 살 수 있다는 인식이 퍼져있어서 주로 노인들이 집에 1~2그램씩 가지고 있다는 것입니다. 뿐만 아니라 빙두가 각종 질환을 고치는데 효과가 있다고 믿는 사람도 있습니다. 마약을 약으로 사용하는 이들은 빙두가 중독되지 않는다고 굳게 믿고 있습니다. 이러한 잘못된 인식은 빙두가 북한 사회에 퍼지는데 결정적인 역할을 하고 있습니다.",
    "북한사람들이 빙두를 하는 이유는 다양합니다. 먼저 의료가 일천한 북한에서는 빙두가 응급처치약으로 쓰이곤 합니다. 뇌혈전이나 일산화탄소 중독으로 쓰러질 경우 빙두를 하면 살 수 있다는 인식이 퍼져있어서 주로 노인들이 집에 1~2그램씩 가지고 있다는 것입니다. 뿐만 아니라 빙두가 각종 질환을 고치는데 효과가 있다고 믿는 사람도 있습니다. 마약을 약으로 사용하는 이들은 빙두가 중독되지 않는다고 굳게 믿고 있습니다. 이러한 잘못된 인식은 빙두가 북한 사회에 퍼지는데 결정적인 역할을 하고 있습니다."
]

# 모델을 동일한 디바이스로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 토크나이저를 사용하여 입력 데이터를 인코딩
input_encodings = tokenizer(input_data, padding=True, truncation=True, return_tensors="pt").to(device)

# 모델에 입력 데이터 전달
model.eval()
with torch.no_grad():
    outputs = model(**input_encodings)

# 예측 결과 확인
logits = outputs.logits
predicted_labels = logits.argmax(dim=1)

# 예측 결과 출력
for i, input_text in enumerate(input_data):
    predicted_label = predicted_labels[i].item()
    print(f"Input: {input_text} - Predicted Label: {valid_label(predicted_label)}")


Input: 아이스작대기 텔레 @blueseal1004 북한산술텔상담후 인증채널 확인! 아이스샘플 훅오는 아이스작대기 블루씰과 상담하세요. 테스트 완료되지 않은 물건 판매 안합니다. 무조건 최상 똥술 절대 판매 안합니다. 알만한 사람은 다 아는 블루씰 무조건 안전! ?? 텔레 @blueseal1004작대기 #아이스 #작대기아이스 #아이스작대기 #빙두 #아이스술 #작대기술 #북한산술 #북한산아이스 #시원한술 #차가운술 #찬술 #케이캔디 #케타민 #암페타민 #후리 #사끼 #원석케이 #작대기하는법 #아이스하는법 #액상대마 #허브액상 #작대기팔아요 #아이스팔아요 #크리스탈술 #얼음술 #코카인 #필로폰 #아이스판매 #아이스구매 #아이스후기 #아이스하는법 #브액 #허브액상 #액상대마 #아이스팔아요 #아이스사는곳 - Predicted Label: 마약
Input: 【텔레 @blueseal1004】아이스판매 24시간상담! 아이스작대기 전국드랍텔상담후 인증채널 확인! 아이스샘플 훅오는 아이스작대기 블루씰과 상담하세요. 테스트 완료되지 않은 물건 판매 안합니다. 무조건 최상 똥술 절대 판매 안합니다. 알만한 사람은 다 아는 블루씰 무조건 안전! ?? 텔레 @blueseal1004작대기 #아이스 #작대기아이스 #아이스작대기 #빙두 #아이스술 #작대기술 #북한산술 #북한산아이스 #시원한술 #차가운술 #찬술 #케이캔디 #케타민 #암페타민 #후리 #사끼 #원석케이 #작대기하는법 #아이스하는법 #작대기팔아요 #아이스팔아요 #크리스탈술 #얼음술 #코카인 #필로폰 #아이스판매 #아이스구매 #아이스후기 #아이스하는법 #브액 #허브액상 #액상대마 #아이스팔아요 #아이스사는곳 #크리스탈얼음술 #눈꽃 #물뽕 - Predicted Label: 마약
Input: 아이스작대기 ??텔레문의?-?@MESSIAH66안전한 다크메시아! 언제나 안전보장 시장통 퀄과는 다른 아이스 똥술 멍술과는 비교불가! 뿅가는 아이스 대,소량 전국드랍 완료! ??텔레문의 - @MESSIAH66작대기 #아이스 #작대기아이스 #아

In [2]:
import torch

# GPU가 사용 가능한지 확인
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. CPU will be used.")
    device = torch.device("cpu")

print("Using device:", device)


CUDA is available. GPU will be used.
Using device: cuda


In [7]:
import subprocess

# nvidia-smi 명령어 실행
result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, text=True)

# 결과 출력
print(result.stdout)


Wed May 29 22:35:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P8             16W /  170W |    1028MiB /  12288MiB |     30%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----